In [ ]:
''''
**Object Detection omilla kuvilla Jetson Nanolle.**


HUOM! GOOGLE COLABIT ON KÄYTÖSSÄ 12 TUNTIA, JOTEN KOVIN SUURTA OTOSTA EI KANNATA ALKAA TEKEMÄÄN.

VAIHDA GOOGLE COLABIN RUNTIMEKSI GPU! Runtime -> Change runtime type -> GPU

Ennen tämän Google Colabin käyttöä:

Ota kuvia esineistä, jotka haluat opettaa (n. 50 kuvaa per esine).
Lataa LabelImg (https://github.com/heartexlabs/labelImg), jonka avulla kuvista tehdään PASCAL VOC - .xml-tiedostot (ohjeet labelimg githubissa).
Lisää rectangle-työkalulla neliöt esineiden kohdalle vasemmasta yläkulmasta oikeaan alakulmaan, laita esineet luokkaan (esim. koira, auto).

Kun .xml-tiedostot on saatu, täytyy tiedostot järjestellä.
Oikea kansiorakenne on seuraava:

  Kansio
 |       |Annotations
 |                   |kuva1.xml
 |                   |...
 |       |ImageSets
 |                   |Main
 |                         |test.txt
 |                         |train.txt
 |                         |trainval.txt
 |                         |val.txt
 |       |JPEGImages
 |                   |kuva1.jpg
 |                   |...
 |       labels.txt

ImageSets/Main/*.txt -filujen selitykset:

Kuvat jaetaan kolmeen osaan (test, train, val).
test.txt tulee n. 5% kuvista, tiedostonimi ilman tiedostopäätettä, yksi yhdelle riville (eli kuva1, ei kuva1.jpg)
val.txt tulee n. 20% kuvista, tiedostonimi ilman tiedostopäätettä
train.txt tulee loppujen kuvien tiedostonimi ilman tiedostopäätettä
trainval.txt laitetaan molempien train.txt ja val.txt sisältö.

Tiedostonimet saa kätevästi Windowsissa komentokehoitteesta menemään oikeaan kansioon
(esim. cd C:\Users\sever\Desktop\modeltiedostot) ja kirjoittamalla dir /s *.jpg >listjpg.txt
Tämän jälkeen voi kun listjpg.txt -tiedoston aukaisee Notepadilla, voi Etsi -> Korvaa 
vaihtaa kaikki .jpg tyhjiksi, jolloin tiedostosta lähtee tiedostopäätteet.

labels.txt tulee esineiden luokat (esim. koira, auto; SAMAT MITÄ LABELIMG-LUOKIKSI TULI),
pelkkä luokka yhdelle riville, esim:
koira
kissa
auto



Helpoin tapa saada kuvat Google Colabin käyttöön on siirtää ne omaan Google Driveen,
joka mountataan myöhemmin koodissa.

Siirrä koko kansio Google Driveen.

Käytetty apuna näitä sivuja:

https://github.com/dusty-nv/jetson-inference/blob/master/docs/pytorch-collect-detection.md
https://github.com/dusty-nv/pytorch-ssd
        
''''

Koonnut: Jesse Severinkangas

Asennetaan PyTorchin tarvitsemat kirjastot

In [1]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116


Kloonataan pytorch-ssd githubista






In [2]:
!git clone https://github.com/dusty-nv/pytorch-ssd

Cloning into 'pytorch-ssd'...
remote: Enumerating objects: 972, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 972 (delta 29), reused 39 (delta 25), pack-reused 919
Receiving objects: 100% (972/972), 1.10 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (635/635), done.


Asennetaan pytorch-ssd tarvitsemat kirjastot

In [3]:
!pip3 install -r pytorch-ssd/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Mountataan Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Siirrytään pytorch-ssd -kansioon.

In [5]:
%cd /content/pytorch-ssd/

/content/pytorch-ssd


Ladataan valmis model

In [6]:
!wget https://nvidia.box.com/shared/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth -O models/mobilenet-v1-ssd-mp-0_675.pt

--2023-02-26 19:21:01--  https://nvidia.box.com/shared/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth
Resolving nvidia.box.com (nvidia.box.com)... 74.112.186.144
Connecting to nvidia.box.com (nvidia.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth [following]
--2023-02-26 19:21:02--  https://nvidia.box.com/public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth
Reusing existing connection to nvidia.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://nvidia.app.box.com/public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth [following]
--2023-02-26 19:21:02--  https://nvidia.app.box.com/public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth
Resolving nvidia.app.box.com (nvidia.app.box.com)... 74.112.186.144
Connecting to nvidia.app.box.com (nvidia.app.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

Koodi trainingin aloittamiseen.<p>
Muuta --data -argumenttiin kansio, jossa datasi on (tässä MyDrivessä oleva esprasp/ -kansio).<p>
Voit muuttaa --model-dir -argumenttiin, mihin haluat modelin tallentuvan.<p>
Epochien määrää muutetaan argumentilla --num-epochs=30



In [7]:
!python3 train_ssd.py --dataset-type=voc --data=/content/gdrive/MyDrive/esprasp/ --model-dir=/content/pytorch-ssd/models/oma/ --pretrained-ssd=/content/pytorch-ssd/models/mobilenet-v1-ssd-mp-0_675.pt

2023-02-26 19:23:45.163486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 19:23:47.069631: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-26 19:23:47.069777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist

Voit ladata modelin itselle pakkaamalla sen

In [9]:
!zip -r /content/model.zip /content/pytorch-ssd/models/oma

  adding: content/pytorch-ssd/models/oma/ (stored 0%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-18-Loss-1.5261343717575073.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-11-Loss-2.3822885751724243.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/labels.txt (stored 0%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-28-Loss-1.735278844833374.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-19-Loss-1.753073275089264.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-25-Loss-1.9126324653625488.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-3-Loss-2.332645893096924.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-12-Loss-2.5792912244796753.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-16-Loss-1.737638533115387.pth (deflated 6%)
  adding: content/pytorch-ssd/models/oma/mb1-ssd-Epoch-29-Loss-1.125849962234497.pth (d

Ladataan tiedosto komennolla:

In [10]:
from google.colab import files
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Muutetaan PyTorch-model .onnx-muotoon. Tämä ei aina toimi, sillä tarvii Google Colabista CUDA-näytönohjaimen, joita ei ole aina vapaana.

In [8]:
!python3 onnx_export.py --model-dir=models/oma

Namespace(batch_size=1, height=300, input='', labels='labels.txt', model_dir='models/oma', net='ssd-mobilenet', output='', width=300)
=> running on device cpu
=> found best checkpoint with loss 1.125849962234497 (models/oma/mb1-ssd-Epoch-29-Loss-1.125849962234497.pth)
creating network:  ssd-mobilenet
num classes:       3
=> loading checkpoint:  models/oma/mb1-ssd-Epoch-29-Loss-1.125849962234497.pth
Traceback (most recent call last):
  File "onnx_export.py", line 95, in <module>
    dummy_input = torch.randn(args.batch_size, 3, args.height, args.width).cuda()
  File "/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py", line 229, in _lazy_init
    torch._C._cuda_init()
RuntimeError: No CUDA GPUs are available


Modelin voi ladata etsimällä sen sivupalkista /content/pytorch-ssd, painamalla hiiren toistanappia ja Download.

Modelin voi muuttaa .onnx -tiedostoksi myös jetson nanossa. Siirrä models.zip jetsoniin, pura /jetson-inference/python/training/detection/ssd/models -kansioon ja muuta komennolla:


```
$ python3 onnx_export.py --model-dir=models/oma
```

onnx_export.py löytyy kansiosta jetson-inference/python/training/detection/ssd/


Modelia voi käyttää esim. detectnetillä komennolla

```
detectnet --model=$NET/ssd-mobilenet.onnx --labels=$NET/labels.txt \
          --input-blob=input_0 --output-cvg=scores --output-bbox=boxes \
            csi://0
```

